<a href="https://colab.research.google.com/github/Maximi652/efficient-slm-architectures/blob/main/efficient_slm_architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio
!pip install torch
!pip install transformers
!pip install -q huggingface_hub

In [ ]:
import json

# Open the file in read mode ('r')
with open("/Training_Data/12B_combined_golden.json", 'r') as f:
    # Now you can load the JSON data from the file object 'f'
    training_data = json.load(f)

# print(json.dumps(training_data['questions'][0], indent=2))

In [8]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

# Pfad zu deinem lokal gespeicherten Modell
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    "/content/localQwen2.5-7B-Instruct",
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

# Antwortfunktion
def chat(user_input, history):
    history = history or []

    # Chat-Verlauf vorbereiten
    messages = [{"role": "system", "content": "You are a helpful assistant."}]
    for user, bot in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": bot})
    messages.append({"role": "user", "content": user_input})

    # Eingabe vorbereiten
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

    # Antwort generieren
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_new_tokens=1024,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(output[0][input_ids.shape[-1]:], skip_special_tokens=True)
    history.append((user_input, response))
    return "", history

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Qwen 2.5 7B Instruct Chatbot")
    chatbot = gr.Chatbot()
    state = gr.State([])
    msg = gr.Textbox(label="Deine Nachricht", placeholder="Frag mich etwas...", lines=2)
    clear = gr.Button("🧹 Verlauf löschen")

    msg.submit(chat, [msg, state], [msg, chatbot])
    clear.click(lambda: ([], []), None, [chatbot, state])

demo.launch()


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [4]:
# nach dem Testen ggf. den Server schließen
demo.close()

Closing server running on port: 7860


In [2]:
from huggingface_hub import snapshot_download

# Zielverzeichnis
target_dir = "./localQwen2.5-7B-Instruct"

# Lade das Modell direkt von Hugging Face in diesen Ordner (spart RAM!)
snapshot_download(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    local_dir=target_dir,
    local_dir_use_symlinks=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

LICENSE:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

'/content/localQwen2.5-7B-Instruct'